In [1]:
%matplotlib inline

import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import time
from torchvision.models import resnet50, ResNet50_Weights, ResNet152_Weights, EfficientNet_B7_Weights

/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
#Select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [4]:
#Dataset location
TRAIN_DATA_PATH = "../Data/CATS_DOGS/train"
TEST_DATA_PATH = "../Data/CATS_DOGS/test"

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    torchvision.datasets.ImageFolder(TRAIN_DATA_PATH, data_transforms['train']),
    'validation': 
    torchvision.datasets.ImageFolder(TEST_DATA_PATH, data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=16),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=16)  # for Kaggle
}

In [5]:
def test(model, data_loader):
    #Sets the module in evaluation mode
    model.eval()
    test_loss = 0
    correct = 0
    #correct.to(device)
    
    #dont update dynamic computation graph
    with torch.no_grad():
        #for every example in test
        for data, target in data_loader:
            
            target = target.view(-1, 1).float()
            target.to(device)
            
            #evaluate the model
            output = model(data.to(device))
            
            #acumulate the loss
            test_loss += F.binary_cross_entropy(output, target.to(device)).item()
            
            pred_cls = output.round()
            correct += pred_cls.eq(target.view(-1, 1).to(device)).sum() 
        
    test_loss /= len(data_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(data_loader.dataset), 100. * correct / len(data_loader.dataset)))

In [6]:
def train(model, data_loader, test_on_every_epoch = False):
    
    for epoch in range(1, num_epochs+1):
    
        
        if test_on_every_epoch:
            test()
        
        start = time.time()
  
        for batch_idx, (data, target) in enumerate(data_loader):
        
            model.train()
        
            #forward pass
            out = model(data.to(device))
        
            #Use negative log likelihood loss.
            loss = criterion(out, target.view(-1, 1).float().to(device))
        
        
            #with this gradients are calculated
            loss.backward()
    
            #update gradients
            optimizer.step()
        
            #Set gradients to zero
            optimizer.zero_grad()

            #Display iteration statistics
            if batch_idx % log_interval == 0:
            
                #print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(data_loader.dataset),100. * batch_idx / len(data_loader), loss.item()))
    
        end = time.time()
        print('Time: {} '.format(end - start))
    

### Test ResNet50

In [28]:
# Hyper parameters
num_epochs = 4
#batchsize = 32
#batchsize_test = 32
learning_rate = 0.01
momentum = 0.5
log_interval = 60

#Network model
model = resnet50(weights="IMAGENET1K_V1").to(device)
#Disable training for all parameters in pretrained model
for param in model.parameters():
    param.requires_grad = False   

#Change last fully conected layer to a one with 1 outputs. This layer is trainable.
model.fc = torch.nn.Sequential(
               torch.nn.Linear(2048, 128),
               torch.nn.ReLU(inplace=True),
               torch.nn.Linear(128, 1),
               torch.nn.Sigmoid()).to(device)


#Stochastic gradient decent
optimizer = optim.SGD(model.parameters(), lr=learning_rate,momentum=momentum)
#Binary cross entropy loss
criterion = torch.nn.BCELoss()

train_losses = []
train_counter = []
test_losses = []

In [17]:
#just for testing
it = iter(dataloaders['train'])
data, target = next(it)
data.shape

out = model(data)

In [18]:
out.shape

torch.Size([32, 1])

In [34]:
train(model, dataloaders['train'])

Train Epoch: 1 [0/18743 (0%)]	Loss: 2.174237


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 1 [1920/18743 (10%)]	Loss: 0.141219
Train Epoch: 1 [3840/18743 (20%)]	Loss: 0.130228
Train Epoch: 1 [5760/18743 (31%)]	Loss: 0.098233
Train Epoch: 1 [7680/18743 (41%)]	Loss: 0.031510
Train Epoch: 1 [9600/18743 (51%)]	Loss: 0.111414
Train Epoch: 1 [11520/18743 (61%)]	Loss: 0.051351
Train Epoch: 1 [13440/18743 (72%)]	Loss: 0.027579
Train Epoch: 1 [15360/18743 (82%)]	Loss: 0.114412
Train Epoch: 1 [17280/18743 (92%)]	Loss: 0.070859
Time: 27.14565110206604 
Train Epoch: 2 [0/18743 (0%)]	Loss: 0.028978
Train Epoch: 2 [1920/18743 (10%)]	Loss: 0.022781


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 2 [3840/18743 (20%)]	Loss: 0.046417
Train Epoch: 2 [5760/18743 (31%)]	Loss: 0.243914
Train Epoch: 2 [7680/18743 (41%)]	Loss: 0.009209
Train Epoch: 2 [9600/18743 (51%)]	Loss: 0.015112
Train Epoch: 2 [11520/18743 (61%)]	Loss: 0.002013
Train Epoch: 2 [13440/18743 (72%)]	Loss: 0.152536
Train Epoch: 2 [15360/18743 (82%)]	Loss: 0.135802
Train Epoch: 2 [17280/18743 (92%)]	Loss: 0.167618
Time: 27.12320303916931 
Train Epoch: 3 [0/18743 (0%)]	Loss: 0.007859
Train Epoch: 3 [1920/18743 (10%)]	Loss: 0.045653
Train Epoch: 3 [3840/18743 (20%)]	Loss: 0.065216
Train Epoch: 3 [5760/18743 (31%)]	Loss: 0.013973
Train Epoch: 3 [7680/18743 (41%)]	Loss: 0.020603
Train Epoch: 3 [9600/18743 (51%)]	Loss: 0.242618
Train Epoch: 3 [11520/18743 (61%)]	Loss: 0.018440
Train Epoch: 3 [13440/18743 (72%)]	Loss: 0.022802
Train Epoch: 3 [15360/18743 (82%)]	Loss: 0.016636


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 3 [17280/18743 (92%)]	Loss: 0.053994
Time: 27.060869216918945 
Train Epoch: 4 [0/18743 (0%)]	Loss: 0.060914
Train Epoch: 4 [1920/18743 (10%)]	Loss: 0.015744
Train Epoch: 4 [3840/18743 (20%)]	Loss: 0.060671
Train Epoch: 4 [5760/18743 (31%)]	Loss: 0.010037
Train Epoch: 4 [7680/18743 (41%)]	Loss: 0.053991
Train Epoch: 4 [9600/18743 (51%)]	Loss: 0.025629
Train Epoch: 4 [11520/18743 (61%)]	Loss: 0.034511


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 4 [13440/18743 (72%)]	Loss: 0.037606
Train Epoch: 4 [15360/18743 (82%)]	Loss: 0.049939
Train Epoch: 4 [17280/18743 (92%)]	Loss: 0.259232
Time: 26.845443964004517 


In [35]:
test(model, dataloaders['validation'])


Test set: Avg. loss: 0.0018, Accuracy: 6124/6251 (98%)



### So?

In notebook 16, I used a simpler model, same training data and validation set, same hyperparameters and same optimizer and criterion function, ran for 4 epochs and got a 76% of acuracy.
Using pretrained ResNet50 model, just changing a little the last layers, which are the only trainable by the way, I was able to get 98% of acuracy in just 4 epochs. 

### Test Resnet152

In [13]:
# Hyper parameters
num_epochs = 4
#batchsize = 32
#batchsize_test = 32
learning_rate = 0.01
momentum = 0.5
log_interval = 60

#Network model
model = torchvision.models.resnet152(weights=ResNet152_Weights.IMAGENET1K_V2).to(device)
#Disable training for all parameters in pretrained model
for param in model.parameters():
    param.requires_grad = False   

#Change last fully conected layer to a one with 1 outputs. This layer is trainable.
model.fc = torch.nn.Sequential(
               torch.nn.Linear(2048, 128),
               torch.nn.ReLU(inplace=True),
               torch.nn.Linear(128, 1),
               torch.nn.Sigmoid()).to(device)


#Stochastic gradient decent
optimizer = optim.SGD(model.parameters(), lr=learning_rate,momentum=momentum)
#Binary cross entropy loss
criterion = torch.nn.BCELoss()

train_losses = []
train_counter = []
test_losses = []

In [14]:
train(model, dataloaders['train'])

Train Epoch: 1 [0/18743 (0%)]	Loss: 0.687897
Train Epoch: 1 [1920/18743 (10%)]	Loss: 0.379074
Train Epoch: 1 [3840/18743 (20%)]	Loss: 0.135129
Train Epoch: 1 [5760/18743 (31%)]	Loss: 0.135480
Train Epoch: 1 [7680/18743 (41%)]	Loss: 0.064524
Train Epoch: 1 [9600/18743 (51%)]	Loss: 0.043436


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 1 [11520/18743 (61%)]	Loss: 0.072983
Train Epoch: 1 [13440/18743 (72%)]	Loss: 0.042779
Train Epoch: 1 [15360/18743 (82%)]	Loss: 0.076443
Train Epoch: 1 [17280/18743 (92%)]	Loss: 0.060013
Time: 58.462249517440796 
Train Epoch: 2 [0/18743 (0%)]	Loss: 0.050965
Train Epoch: 2 [1920/18743 (10%)]	Loss: 0.044757
Train Epoch: 2 [3840/18743 (20%)]	Loss: 0.107943
Train Epoch: 2 [5760/18743 (31%)]	Loss: 0.036360
Train Epoch: 2 [7680/18743 (41%)]	Loss: 0.023734


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 2 [9600/18743 (51%)]	Loss: 0.065608
Train Epoch: 2 [11520/18743 (61%)]	Loss: 0.036655
Train Epoch: 2 [13440/18743 (72%)]	Loss: 0.112597
Train Epoch: 2 [15360/18743 (82%)]	Loss: 0.030062
Train Epoch: 2 [17280/18743 (92%)]	Loss: 0.024547
Time: 58.34886598587036 
Train Epoch: 3 [0/18743 (0%)]	Loss: 0.012428
Train Epoch: 3 [1920/18743 (10%)]	Loss: 0.249961
Train Epoch: 3 [3840/18743 (20%)]	Loss: 0.010485
Train Epoch: 3 [5760/18743 (31%)]	Loss: 0.036099
Train Epoch: 3 [7680/18743 (41%)]	Loss: 0.039868
Train Epoch: 3 [9600/18743 (51%)]	Loss: 0.030681
Train Epoch: 3 [11520/18743 (61%)]	Loss: 0.055848


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 3 [13440/18743 (72%)]	Loss: 0.049777
Train Epoch: 3 [15360/18743 (82%)]	Loss: 0.004573
Train Epoch: 3 [17280/18743 (92%)]	Loss: 0.030309
Time: 58.48272967338562 
Train Epoch: 4 [0/18743 (0%)]	Loss: 0.002478
Train Epoch: 4 [1920/18743 (10%)]	Loss: 0.039159
Train Epoch: 4 [3840/18743 (20%)]	Loss: 0.007415
Train Epoch: 4 [5760/18743 (31%)]	Loss: 0.028232
Train Epoch: 4 [7680/18743 (41%)]	Loss: 0.034050
Train Epoch: 4 [9600/18743 (51%)]	Loss: 0.067180
Train Epoch: 4 [11520/18743 (61%)]	Loss: 0.216953
Train Epoch: 4 [13440/18743 (72%)]	Loss: 0.010488


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 4 [15360/18743 (82%)]	Loss: 0.027229
Train Epoch: 4 [17280/18743 (92%)]	Loss: 0.153607
Time: 58.76094079017639 


In [15]:
test(model, dataloaders['validation'])


Test set: Avg. loss: 0.0011, Accuracy: 6191/6251 (99%)



### So?

For ResNet152, using the same approach as in the last example, we got a 99% of accuracy.

### Test EfficientNet b7

In [7]:
EfficientNet_B7_Weights.IMAGENET1K_V1.transforms

functools.partial(<class 'torchvision.transforms._presets.ImageClassification'>, crop_size=600, resize_size=600, interpolation=<InterpolationMode.BICUBIC: 'bicubic'>)

In [7]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((600,600)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((600,600)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    torchvision.datasets.ImageFolder(TRAIN_DATA_PATH, data_transforms['train']),
    'validation': 
    torchvision.datasets.ImageFolder(TEST_DATA_PATH, data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=16,
                                shuffle=True,
                                num_workers=16),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=16,
                                shuffle=False,
                                num_workers=16)  # for Kaggle
}

In [10]:
it = iter(dataloaders['train'])
data, target = next(it)
data.shape

out = model(data.to(device))

In [13]:
model.classifier

Sequential(
  (0): Dropout(p=0.5, inplace=True)
  (1): Linear(in_features=2560, out_features=1000, bias=True)
)

In [14]:
# Hyper parameters
num_epochs = 4
#batchsize = 32
#batchsize_test = 32
learning_rate = 0.01
momentum = 0.5
log_interval = 60

#Network model
model = torchvision.models.efficientnet_b7(weights=EfficientNet_B7_Weights.IMAGENET1K_V1).to(device)
#Disable training for all parameters in pretrained model
for param in model.parameters():
    param.requires_grad = False   

#Change last fully conected layer to a one with 1 outputs. This layer is trainable.
model.classifier = torch.nn.Sequential(
               torch.nn.Linear(2560, 128),
               torch.nn.ReLU(inplace=True),
               torch.nn.Linear(128, 1),
               torch.nn.Sigmoid()).to(device)


#Stochastic gradient decent
optimizer = optim.SGD(model.parameters(), lr=learning_rate,momentum=momentum)
#Binary cross entropy loss
criterion = torch.nn.BCELoss()

train_losses = []
train_counter = []
test_losses = []

In [15]:
train(model, dataloaders['train'])

Train Epoch: 1 [0/18743 (0%)]	Loss: 0.695592
Train Epoch: 1 [960/18743 (5%)]	Loss: 0.639343
Train Epoch: 1 [1920/18743 (10%)]	Loss: 0.458206
Train Epoch: 1 [2880/18743 (15%)]	Loss: 0.341754
Train Epoch: 1 [3840/18743 (20%)]	Loss: 0.203342
Train Epoch: 1 [4800/18743 (26%)]	Loss: 0.158357
Train Epoch: 1 [5760/18743 (31%)]	Loss: 0.196246
Train Epoch: 1 [6720/18743 (36%)]	Loss: 0.120954
Train Epoch: 1 [7680/18743 (41%)]	Loss: 0.168544
Train Epoch: 1 [8640/18743 (46%)]	Loss: 0.046757
Train Epoch: 1 [9600/18743 (51%)]	Loss: 0.060458
Train Epoch: 1 [10560/18743 (56%)]	Loss: 0.065150
Train Epoch: 1 [11520/18743 (61%)]	Loss: 0.142871
Train Epoch: 1 [12480/18743 (67%)]	Loss: 0.080331
Train Epoch: 1 [13440/18743 (72%)]	Loss: 0.036240
Train Epoch: 1 [14400/18743 (77%)]	Loss: 0.047595


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 1 [15360/18743 (82%)]	Loss: 0.093751
Train Epoch: 1 [16320/18743 (87%)]	Loss: 0.031336
Train Epoch: 1 [17280/18743 (92%)]	Loss: 0.041792
Train Epoch: 1 [18240/18743 (97%)]	Loss: 0.031712
Time: 541.9774088859558 
Train Epoch: 2 [0/18743 (0%)]	Loss: 0.006529
Train Epoch: 2 [960/18743 (5%)]	Loss: 0.039117
Train Epoch: 2 [1920/18743 (10%)]	Loss: 0.016267
Train Epoch: 2 [2880/18743 (15%)]	Loss: 0.277881
Train Epoch: 2 [3840/18743 (20%)]	Loss: 0.137085
Train Epoch: 2 [4800/18743 (26%)]	Loss: 0.267798
Train Epoch: 2 [5760/18743 (31%)]	Loss: 0.037081
Train Epoch: 2 [6720/18743 (36%)]	Loss: 0.054386
Train Epoch: 2 [7680/18743 (41%)]	Loss: 0.048425
Train Epoch: 2 [8640/18743 (46%)]	Loss: 0.059025
Train Epoch: 2 [9600/18743 (51%)]	Loss: 0.009134
Train Epoch: 2 [10560/18743 (56%)]	Loss: 0.023747
Train Epoch: 2 [11520/18743 (61%)]	Loss: 0.013351
Train Epoch: 2 [12480/18743 (67%)]	Loss: 0.017892
Train Epoch: 2 [13440/18743 (72%)]	Loss: 0.011756
Train Epoch: 2 [14400/18743 (77%)]	Loss: 0

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 2 [15360/18743 (82%)]	Loss: 0.125342
Train Epoch: 2 [16320/18743 (87%)]	Loss: 0.289530
Train Epoch: 2 [17280/18743 (92%)]	Loss: 0.061905
Train Epoch: 2 [18240/18743 (97%)]	Loss: 0.004642
Time: 543.5254535675049 
Train Epoch: 3 [0/18743 (0%)]	Loss: 0.079853


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 3 [960/18743 (5%)]	Loss: 0.037287
Train Epoch: 3 [1920/18743 (10%)]	Loss: 0.006270
Train Epoch: 3 [2880/18743 (15%)]	Loss: 0.007050
Train Epoch: 3 [3840/18743 (20%)]	Loss: 0.072400
Train Epoch: 3 [4800/18743 (26%)]	Loss: 0.071890
Train Epoch: 3 [5760/18743 (31%)]	Loss: 0.012660
Train Epoch: 3 [6720/18743 (36%)]	Loss: 0.079342
Train Epoch: 3 [7680/18743 (41%)]	Loss: 0.031960
Train Epoch: 3 [8640/18743 (46%)]	Loss: 0.041830
Train Epoch: 3 [9600/18743 (51%)]	Loss: 0.036380
Train Epoch: 3 [10560/18743 (56%)]	Loss: 0.182501
Train Epoch: 3 [11520/18743 (61%)]	Loss: 0.042463
Train Epoch: 3 [12480/18743 (67%)]	Loss: 0.005602
Train Epoch: 3 [13440/18743 (72%)]	Loss: 0.023093
Train Epoch: 3 [14400/18743 (77%)]	Loss: 0.009413
Train Epoch: 3 [15360/18743 (82%)]	Loss: 0.098556
Train Epoch: 3 [16320/18743 (87%)]	Loss: 0.041606
Train Epoch: 3 [17280/18743 (92%)]	Loss: 0.062724
Train Epoch: 3 [18240/18743 (97%)]	Loss: 0.052399
Time: 570.105890750885 
Train Epoch: 4 [0/18743 (0%)]	Loss: 0.

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 4 [9600/18743 (51%)]	Loss: 0.216792
Train Epoch: 4 [10560/18743 (56%)]	Loss: 0.006671
Train Epoch: 4 [11520/18743 (61%)]	Loss: 0.007289
Train Epoch: 4 [12480/18743 (67%)]	Loss: 0.004812
Train Epoch: 4 [13440/18743 (72%)]	Loss: 0.034706
Train Epoch: 4 [14400/18743 (77%)]	Loss: 0.016296
Train Epoch: 4 [15360/18743 (82%)]	Loss: 0.042554
Train Epoch: 4 [16320/18743 (87%)]	Loss: 0.044532
Train Epoch: 4 [17280/18743 (92%)]	Loss: 0.063566
Train Epoch: 4 [18240/18743 (97%)]	Loss: 0.002083
Time: 550.1512305736542 


In [16]:
test(model, dataloaders['validation'])


Test set: Avg. loss: 0.0018, Accuracy: 6208/6251 (99%)



### ToDo

- NA

### Resources

- https://www.kaggle.com/code/pmigdal/transfer-learning-with-resnet-50-in-pytorch
- https://paperswithcode.com/sota/image-classification-on-imagenet
- https://pytorch.org/vision/stable/models.html